# Integrated Intelligent Systems: Module 8 - Knowledge and Reasoning (Tutorial)

This notebook introduces the fundamentals of knowledge/reasoning and basic practices of it.

## Part 1: Concepts and Definitions

In [4]:
from IPython.display import display, HTML,IFrame
display(IFrame('krr_quiz.html', width=1000, height=1350))

## Part 2: Hands-On (Prolog - Logic Programming)

To prepare you for the Autonomous Navigate-to-Grasp Challenge, these two Python-based exercises focus on integrating Prolog (via PySwip) into the robot's "Sense-Think-Act" loop.

![Logic Programming](imgs/program_sample.png "Logic Programming" )

### Exercise 2.1: Knowledge Representation

In this exercise, we investigate the representation of an environment map.

In [1]:
from pyswip import Prolog

prolog = Prolog()

def representing():
    # 1. ASSERTING SPATIAL PROPERTIES
    
    # Room: Size (X, Y, Z) based on Task Specification
    # The room is a 10m x 10m x 10m cube
    prolog.assertz("room_size(10.0, 10.0, 10.0)") #
    
    # Robot: Initial Position (x, y) and Size (radius for collision)
    # Positions are randomized per execution; using (1.0, 1.0) as an example
    prolog.assertz("robot_pose(1.0, 1.0)")
    prolog.assertz("robot_size(0.3)") # Assume a footprint radius of 0.3m
    
    # Table: Position (x,y) and Dimensions (width, depth)
    prolog.assertz("object_pose(table, 5.0, 5.0)") # Randomized center
    prolog.assertz("object_size(table, 1.5, 0.8)") #
    
    # Target Object: Red Cylinder (r=0.04, h=0.12)
    prolog.assertz("object_pose(target_object, 7.0, 7.0)")
    prolog.assertz("object_size(target_object, 0.08, 0.12)") #
    
    # Obstacle: Blue Cube (0.4m side)
    prolog.assertz("object_pose(obstacle_1, 3.0, 3.0)")
    prolog.assertz("object_size(obstacle_1, 0.4, 0.4)") #
    
    # 2. DEFINING REASONING RULES
    
    # Safety Rule: Ensure a position (X, Y) is inside the 10x10 room boundaries
    prolog.assertz("""
        is_within_room(X, Y) :-
            room_size(MaxX, MaxY, _),
            X >= 0, X =< MaxX,
            Y >= 0, Y =< MaxY
    """)
    
    # Collision Rule: The robot does not collide with any obstacle
    prolog.assertz("""
        collision_alert(RX, RY) :-
            % Get the robot's physical radius
            robot_size(R_Radius),
            
            % Find any obstacle and its size in the environment
            object_pose(Obstacle, OX, OY),
            atom_concat(obstacle_, _, Obstacle), % Ensure we are checking an obstacle
            object_size(Obstacle, S, _),
            
            % Treat the square obstacle as a circle with radius O_Radius
            % Using S/2 for an inscribed circle or S * 0.707 for circumscribed
            O_Radius is S / 2,
            
            % Calculate Euclidean distance between centers
            DX is RX - OX,
            DY is RY - OY,
            Distance is sqrt(DX*DX + DY*DY),
            
            % Collision occurs if distance < sum of both radii
            SafeDistance is R_Radius + O_Radius,
            Distance =< SafeDistance
    """)

    # Rule for proximity: Check if the robot is within reach of an object
    prolog.assertz("""
         within_reach(RobotX, RobotY, Obj) :-
              object_pose(Obj, ObjX, ObjY),
              DX is RobotX - ObjX,
              DY is RobotY - ObjY,
              Distance is sqrt(DX*DX + DY*DY),
              Distance =< 1.0
    """)

    # Rule for 'On Table': Check if an object's coordinates fall within table boundaries
    prolog.assertz("""
        is_on_table(Obj) :-
            object_pose(Obj, OX, OY),
            object_pose(table, TX, TY),
            object_size(table, TW, TD),
            OX >= (TX - TW/2), OX =< (TX + TW/2),
            OY >= (TY - TD/2), OY =< (TY + TD/2)
    """)
    
    # Note: No trailing period inside the string for PySwip
    prolog.assertz("""
       distance(X1, Y1, X2, Y2, Dist) :-
        DX is X1 - X2,
        DY is Y1 - Y2,
        Dist is sqrt(DX*DX + DY*DY)
    """)

    

def update_robot_pose(new_x, new_y):
    # 1. Remove ALL existing robot_pose facts to prevent multiple positions
    prolog.retractall("robot_pose(_, _)")

    # 2. Assert the new position
    prolog.assertz(f"robot_pose({new_x}, {new_y})")
    

    
def path_planning():
    # Helper to define the grid size based on robot_size (0.3m radius -> 0.6m cell)
    prolog.assertz("grid_step(0.6)")

    # 0. Rule: Is a specific cell (X, Y) obstructed?
    # This checks if the cell coordinates would trigger a collision_alert
    prolog.assertz("""
        is_obstructed(X, Y) :-
            collision_alert(X, Y)
    """)

    # Rule: Is a cell valid? (In room and not obstructed)
    prolog.assertz("""
        is_free(X, Y) :-
            is_within_room(X, Y),
            \\+ is_obstructed(X, Y)
    """)
    

    # 1. Define possible movements
    prolog.assertz("step_size(0.6)")
    prolog.assertz("move(X, Y, NX, Y, S) :- NX is X + S")
    prolog.assertz("move(X, Y, NX, Y, S) :- NX is X - S")
    prolog.assertz("move(X, Y, X, NY, S) :- NY is Y + S")
    prolog.assertz("move(X, Y, X, NY, S) :- NY is Y - S")
    

    # 2. Add a helper to pick the next best coordinate
    prolog.assertz("""
        best_next_step(X, Y, TX, TY, Visited, NX, NY) :-
            Step is 0.6,
            % Find all valid neighboring cells
            findall(Dist-CX-CY, (
                move(X, Y, CX, CY, Step),
                is_free(CX, CY),
                \\+ member([CX, CY], Visited),
                distance(CX, CY, TX, TY, Dist)
            ), Candidates),
            % Sort candidates by distance (lowest first)
            sort(Candidates, [_-NX-NY|_])
    """)

    # 3. Simplified, directed pathfinding rule
    prolog.assertz("""
        find_path(X, Y, TX, TY, _, [[X, Y]]) :-
            is_at_goal(X, Y, TX, TY)
    """)

    prolog.assertz("""
        find_path(X, Y, TX, TY, Visited, [[X, Y] | Rest]) :-
            best_next_step(X, Y, TX, TY, Visited, NX, NY),
            find_path(NX, NY, TX, TY, [[NX, NY] | Visited], Rest)
    """)
    
    # Note: Use a threshold (0.6) equal to or slightly larger than your step size
    prolog.assertz("""
        is_at_goal(X, Y, TX, TY) :-
            distance(X, Y, TX, TY, Dist),
            Dist =< 0.6
    """)

    
representing()
path_planning()

### Exercise 2.2: Reasoning

In this exercise, we reason on the basis of our encoded environment map to infer critical knowledge for the robot.

In [2]:
def reasoning(robot_x, robot_y):
    
    # Safety check: Is the robot's current position within the room?
    robot_pos = list(prolog.query("is_within_room("+str(robot_x)+", "+str(robot_y)+")"))
    if robot_pos:
         print(f"Robot is safely at ({robot_x}, {robot_y}) within room boundaries.")
    else:
         print("Safety Alert: Robot is out of bounds!")
            
    
    # Collision check: Is the robot colliding with an obstacle?
    collision = list(prolog.query(f"collision_alert({robot_x}, {robot_y})"))
    
    if collision:
        print ("CRITICAL: Path Obstructed! Re-planning required.")
    else:
        print ("PATH_CLEAR: Proceeding to coordinates.")
    
    
    # Determine if the target is reachable and correctly placed
    reachable = list(prolog.query(f"within_reach({robot_x}, {robot_y}, target_object)"))
    on_table = list(prolog.query("is_on_table(target_object)"))
    
    if reachable and on_table:
        return "READY_TO_GRASP"
    elif on_table:
        print ("NAVIGATE_TO_TABLE")
    else:
        print ("REPLAN_SEARCH")
    

def plan_route(start_x, start_y, goal_x, goal_y):
    # Query: find_path(Start, Goal, InitialVisited, Path)
    query = f"find_path({start_x}, {start_y}, {goal_x}, {goal_y}, [[{start_x},{start_y}]], Path)"
    
    try:
        results = list(prolog.query(query))
        if results:
            print( results[0]["Path"])
        else:
            print("NO PATH FOUND!!!")
    except Exception as e:
        print(f"Prolog Error: {e}")
        return None
    
    
reasoning(15.0,15.0)
update_robot_pose(6.0, 6.0)


Safety Alert: Robot is out of bounds!
PATH_CLEAR: Proceeding to coordinates.
REPLAN_SEARCH


In [3]:
plan_route(8.0, 8.0, 5.0,5.0)

[[8.0, 8.0], [7.4, 8.0], [7.4, 7.4], [6.800000000000001, 7.4], [6.800000000000001, 6.800000000000001], [6.200000000000001, 6.800000000000001], [6.200000000000001, 6.200000000000001], [5.600000000000001, 6.200000000000001], [5.600000000000001, 5.600000000000001], [5.000000000000002, 5.600000000000001], [5.000000000000002, 5.000000000000002]]
